In [55]:
import pandas as pd

common_fields = [
    "passCmp",
    "passAtt",
    "passYds",
    "passTd",
    "passInt",
    #'passLong',
    #'passRating',
    #'passTargetYds',
    "rushAtt",
    "rushYds",
    "rushTd",
    #'rushLong',
    #'rushYdsBc',
    #'rushYdsAc',
    #'rushBrokenTackles',
    "recAtt",
    "recCmp",
    "recYds",
    "recTd",
    #'recDrops',
    #'recLong',
    #'recDepth',
    #'recYac',
    "sacked",
    "fumbles",
    "fumblesLost",
]

defensive_df = pd.read_parquet("../cache/parquet/defensive.parquet").sort_values(
    by=["season", "week"], ascending=False
)

# df[df['game_id'] == '201909080car']
# select = df[df['playerDisplay'] == 'Kenneth Gainwell']

# select.sort_values(by=['season','week'])
season_df = defensive_df[defensive_df["season"] == 2024]

teams = (
    season_df.groupby(["team", "week"])
    .sum()
    .groupby(["team"])
    .mean()[common_fields]
    .reset_index()
)
roles = season_df.groupby(["team", "role"]).mean()

means = roles[common_fields].reset_index()  # .loc['qb1'][common_fields]

means

,team,role,passCmp,passAtt,passYds,passTd,passInt,rushAtt,rushYds,rushTd,recAtt,recCmp,recYds,recTd,sacked,fumbles,fumblesLost
0,ARI,qb1,-8.096774,-15.887097,-45.225806,0.478495,-0.747312,1.930108,5.182796,0.580645,-0.010753,-0.010753,-0.021505,0.000000,1.225806,0.516129,0.612903
1,ARI,rb1,0.000000,0.000000,0.000000,0.000000,0.000000,2.172043,7.849462,-0.215054,0.811828,0.629032,11.747312,-0.010753,0.000000,-0.053763,-0.032258
2,ARI,rb2,0.000000,0.000000,0.000000,0.000000,0.000000,3.623656,14.387097,-0.209677,-2.209677,-1.768817,0.118280,0.322581,0.000000,-0.010753,-0.010753
3,ARI,rest,0.000000,0.000000,0.000000,0.000000,0.000000,-1.750000,-14.500000,0.000000,-3.500000,-2.000000,-6.500000,0.000000,0.000000,0.000000,0.000000
4,ARI,wr1,0.000000,0.000000,0.000000,0.000000,0.000000,-0.860215,-4.247312,-0.010753,-7.505376,-3.720430,-25.860215,-0.107527,-0.010753,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,WSH,rb2,0.000000,0.000000,0.000000,0.000000,0.000000,2.091398,25.913978,-0.043011,0.779570,0.220430,0.064516,-0.010753,0.000000,-0.010753,-0.010753
220,WSH,rest,0.000000,0.000000,0.000000,0.000000,0.000000,-0.500000,0.000000,0.000000,-4.750000,-1.250000,-17.750000,0.000000,0.000000,-0.750000,-0.500000
221,WSH,wr1,0.000000,0.000000,0.000000,0.000000,0.000000,-0.172043,-0.806452,-0.010753,3.940860,2.935484,49.876344,1.225806,-0.010753,0.000000,0.000000
222,WSH,wr2,0.000000,0.000000,0.000000,0.000000,0.000000,-0.483871,-5.860215,-0.010753,-3.123656,-1.279570,-5.478495,0.892473,0.000000,0.000000,0.000000


In [51]:
team_df = teams.copy()


def calculate_rating(attempts, completions, yards, touchdowns, turnovers):
    # Positive score means the defense is good, negative is bad

    # if attempts went down, but completions went up, bad for defense
    # attempts up, completions down, good for defense
    score = (attempts - completions) * 3

    if completions < 0:
        score -= completions * 5

    if attempts < 0:
        score -= attempts * 5

    return score + -(+(yards) + (touchdowns * 7) - turnovers * 10)


def calculate_def_rating(player_row):
    return calculate_rating(
        player_row["passAtt"],
        player_row["passCmp"],
        player_row["passYds"],
        player_row["passTd"],
        player_row["passInt"],
    ) + calculate_rating(
        player_row["rushAtt"],
        player_row["rushAtt"],
        player_row["rushYds"],
        player_row["rushTd"],
        player_row["fumblesLost"],
    )


team_df["rating"] = team_df.apply(calculate_def_rating, axis=1)

team_df.sort_values(by=["rating"], ascending=False)

,team,passCmp,passAtt,passYds,passTd,passInt,rushAtt,rushYds,rushTd,recAtt,recCmp,recYds,sacked,fumbles,fumblesLost,rating
16,MIA,-3.908602,-4.489247,-29.666667,-0.354839,0.252688,-10.215054,-37.268817,-0.198925,-5.516129,-4.478495,-38.419355,-0.295699,0.408602,-0.118280,214.553763
17,MIN,5.381720,9.537634,43.935484,-0.177419,1.430108,-13.306452,-80.548387,-0.865591,7.688172,5.612903,49.188172,2.349462,0.741935,-0.118280,202.564516
24,PIT,-3.639785,-4.806452,23.709677,-0.198925,0.252688,-8.489247,-66.709677,-1.043011,-6.510753,-4.241935,22.795699,1.193548,-0.747312,-0.618280,171.661290
10,DET,1.537634,5.247312,1.365591,-1.354839,0.763441,-8.252688,-59.650538,-0.021505,4.118280,0.311828,-10.935484,0.693548,-0.091398,-0.451613,164.693548
26,SEA,-0.462366,0.548387,-57.629032,-0.354839,-0.069892,-6.564516,-20.548387,-0.188172,2.553763,1.290323,-34.446237,1.526882,1.075269,0.215054,154.419355
5,CHI,-1.505376,-0.241935,-31.301075,-0.865591,0.419355,-4.881720,-41.096774,0.290323,-5.413978,-3.930108,-45.763441,-0.306452,-0.069892,0.548387,147.446237
3,BUF,5.537634,7.693548,-47.806452,0.155914,1.086022,-4.908602,-12.784946,-0.688172,5.978495,4.102151,-62.091398,0.693548,-0.080645,0.392473,134.655914
0,ARI,-8.096774,-15.887097,-45.225806,0.478495,-0.747312,5.526882,5.225806,0.134409,-16.236559,-8.698925,-59.801075,1.215054,0.419355,0.559140,130.376344
19,NO,3.381720,5.193548,53.016129,-0.833333,0.919355,-8.940860,-31.333333,-0.188172,0.688172,0.456989,22.376344,1.349462,0.408602,0.381720,93.322581
27,SF,-6.284946,-6.408602,-6.650538,0.134409,-0.080645,-1.844086,2.172043,-0.021505,-3.456989,-3.741935,8.284946,0.370968,0.575269,0.048387,84.903226


In [ ]:
qb_df = means[means["role"] == "qb1"].copy()


def calculate_def_qb_rating(player_row):
    return calculate_rating(
        player_row["passAtt"],
        player_row["passCmp"],
        player_row["passYds"],
        player_row["passTd"],
        player_row["passInt"],
    )


qb_df["rating"] = qb_df.apply(calculate_def_qb_rating, axis=1)

qb_df.sort_values(by=["rating"], ascending=False)

In [56]:
wr1_df = means[means["role"] == "wr1"].copy()
wr2_df = means[means["role"] == "wr2"].copy()
wr3_df = means[means["role"] == "wr3"].copy()


def calculate_wr_rating(player_row):
    return calculate_rating(
        player_row["recAtt"],
        player_row["recCmp"],
        player_row["recYds"],
        player_row["recTd"],
        player_row["fumblesLost"],
    )


wr1_df["rating"] = wr1_df.apply(calculate_wr_rating, axis=1)

wr1_df.sort_values(by=["rating"], ascending=False)

,team,role,passCmp,passAtt,passYds,passTd,passInt,rushAtt,rushYds,rushTd,recAtt,recCmp,recYds,recTd,sacked,fumbles,fumblesLost,rating
4,ARI,wr1,0.0,0.000000,0.0,0.0,0.0,-0.860215,-4.247312,-0.010753,-7.505376,-3.720430,-25.860215,-0.107527,-0.010753,0.000000,0.000000,71.387097
25,BUF,wr1,0.0,0.000000,0.0,0.0,0.0,0.139785,3.086022,-0.010753,-0.215054,-0.731183,-54.134409,-0.607527,-0.010753,0.000000,0.000000,64.666667
67,DEN,wr1,0.0,0.000000,0.0,0.0,0.0,-0.526882,2.419355,-0.010753,-1.139785,-1.564516,-30.107527,-0.107527,-0.010753,0.000000,0.000000,45.655914
60,DAL,wr1,0.0,0.000000,0.0,0.0,0.0,0.838710,2.354839,-0.010753,-2.327957,-1.887097,-22.860215,-0.274194,-0.010753,0.000000,0.000000,44.532258
32,CAR,wr1,0.0,0.000000,0.0,0.0,0.0,-0.193548,-0.913978,-0.010753,-2.193548,-2.731183,-16.306452,-0.274194,-0.010753,0.000000,0.000000,44.462366
214,TEN,wr1,0.0,0.000000,0.0,0.0,0.0,0.306452,-4.580645,-0.177419,-0.193548,-0.741935,-20.478495,-0.274194,-0.010753,-0.166667,0.000000,28.720430
81,GB,wr1,0.0,0.000000,0.0,0.0,0.0,-0.360215,-2.580645,-0.177419,-1.026882,-1.231183,-3.145161,0.059140,-0.010753,0.000000,0.000000,14.634409
88,HOU,wr1,0.0,0.000000,0.0,0.0,0.0,-0.526882,-5.580645,-0.010753,0.983871,0.623656,-9.591398,-0.107527,-0.010753,0.000000,0.000000,11.424731
46,CIN,wr1,0.0,0.000000,0.0,0.0,0.0,-0.193548,-0.913978,-0.010753,-1.870968,-1.408602,2.930108,0.559140,-0.010753,0.000000,0.000000,8.166667
137,NO,wr1,0.0,0.000000,0.0,0.0,0.0,-1.193548,-9.247312,-0.010753,-1.215054,-0.741935,-0.064516,0.069892,-0.010753,0.000000,0.000000,7.940860


In [54]:
rb_df = means[means["role"] == "rb2"].copy()


def calculate_def_rb_rating(player_row):
    return calculate_rating(
        player_row["rushAtt"],
        player_row["rushAtt"],
        player_row["rushYds"],
        player_row["rushTd"],
        player_row["fumblesLost"],
    )


rb_df["rating"] = rb_df.apply(calculate_def_rb_rating, axis=1)

rb_df.sort_values(by=["rating"], ascending=False)

,team,role,passCmp,passAtt,passYds,passTd,passInt,rushAtt,rushYds,rushTd,recAtt,recCmp,recYds,sacked,fumbles,fumblesLost,rating
23,BUF,rb2,0.0,0.0,0.0,0.0,0.0,-5.709677,-33.612903,-0.043011,-1.043011,-0.602151,-3.451613,0.000000,-0.010753,-0.010753,90.903226
170,PIT,rb2,0.0,0.0,0.0,0.0,0.0,-6.532258,-17.446237,-0.043011,-2.220430,-2.112903,-2.602151,0.000000,-0.344086,-0.344086,79.629032
79,GB,rb2,0.0,0.0,0.0,0.0,0.0,-3.548387,-15.637097,-0.064516,-0.032258,0.112903,18.629032,0.000000,-0.016129,-0.016129,51.411290
58,DAL,rb2,0.0,0.0,0.0,0.0,0.0,-3.483871,-8.709677,-0.564516,-1.064516,-0.903226,-9.306452,0.000000,-0.016129,-0.016129,47.338710
16,BAL,rb2,0.0,0.0,0.0,0.0,0.0,-2.741935,-18.752688,0.123656,-2.553763,-1.946237,-18.435484,0.000000,-0.010753,-0.010753,45.198925
72,DET,rb2,0.0,0.0,0.0,0.0,0.0,-2.956989,-15.155914,-0.032258,-1.655914,-1.236559,-8.236559,0.000000,-0.010753,-0.010753,44.844086
86,HOU,rb2,0.0,0.0,0.0,0.0,0.0,-2.043011,-18.747312,0.290323,-0.220430,-0.112903,-4.768817,-0.166667,-0.010753,-0.010753,37.037634
149,NYJ,rb2,0.0,0.0,0.0,0.0,0.0,-2.075269,-16.086022,-0.043011,-0.220430,-0.112903,-0.435484,0.000000,-0.010753,-0.010753,37.032258
114,MIA,rb2,0.0,0.0,0.0,0.0,0.0,-1.010753,-16.989247,-0.043011,-0.354839,-0.258065,-3.827957,0.000000,0.322581,0.322581,30.623656
177,SD,rb2,0.0,0.0,0.0,0.0,0.0,-1.467742,-12.134409,-0.032258,-0.354839,0.419355,-6.892473,0.000000,-0.010753,-0.010753,26.930108
